In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import eel
import plotly.express as px
from io import StringIO

: 

In [ ]:
import xgboost as xgb
import joblib

model = joblib.load('xgb.joblib')

In [ ]:
!pip list

In [ ]:
model

In [ ]:
@eel.expose
def give_res(csv_file: str) -> str:
    df = pd.read_csv(StringIO(csv_file), delimiter=',')
    df['взаимосвязь с водой бин'] = df['взаимосвязь с водой'].apply(lambda x: 1 if x == 'да' else 0)
    for elem in ['болото', 'бугор пучения', 'лес', 'озеро', 'торфяник']:
        df[elem] = df['ландшафт'].apply(lambda x: 1 if x == elem else 0)
    df['взаимосвязь с водой'] = df['взаимосвязь с водой бин'] 
    coord_df = df[['X', 'Y']]
    df = df.drop(columns=['X','Y','ландшафт', 'взаимосвязь с водой бин'])
    df['температура воздуха'] = [x if isinstance(x, float) else 0 for x in df['температура воздуха']]
    df['торф'] = [x if isinstance(x, float) or  isinstance(x, int) else 0 for x in df['торф']]
    y_pred = model.predict(df)
    coord_df['Z'] = y_pred
    color_map = {0: 'red', 1: 'yellow', 2: 'green'}
    types_map = {0: 'нет', 1: 'слив', 2: 'есть'}

    coord_df['color'] = coord_df['Z'].map(color_map)
    coord_df['name'] = coord_df['Z'].map(types_map)
    coord_df['size']=10

    fig = px.scatter_mapbox(coord_df, lat='X', lon='Y', hover_name='name', zoom=10, height=600,color='color', size='size')
    fig.update_layout(mapbox_style='open-street-map', showlegend=False)

    fig.write_html('web/map.html')
    return 'map.html'

In [ ]:
eel.init('web')
eel.start('main.html', size=(1050,1050))

In [ ]:
# Copyright 2023 danilsherki
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

